<a href="https://colab.research.google.com/github/AfshinRezakhani/Thesis1/blob/main/GAN_GRU_d7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Input, Reshape, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

# Load dataset
df = pd.read_csv("/content/d7.csv")
X = df.drop(columns=["F"]).values
y = df["F"].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# GAN parameters
latent_dim = 20

# Generator
def build_generator():
    inputs = Input(shape=(latent_dim,))
    x = Dense(32, activation='relu')(inputs)
    x = Reshape((1, 32))(x)
    x = GRU(64, return_sequences=True)(x)
    x = GRU(64)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(X_train.shape[1])(x)
    return Model(inputs, outputs, name="Generator")

# Discriminator
def build_discriminator():
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(32, activation='relu')(inputs)
    x = Reshape((1, 32))(x)
    x = GRU(64, return_sequences=True)(x)
    x = GRU(64)(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs, name="Discriminator")

# Compile GAN
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False
z = Input(shape=(latent_dim,))
generated_data = generator(z)
disc_output = discriminator(generated_data)
gan = Model(z, disc_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Train GAN
def train_gan(epochs=1500, batch_size=64):
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_samples = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_samples = generator.predict(noise)

        X_combined = np.vstack((real_samples, fake_samples))
        y_combined = np.hstack((np.ones(batch_size), np.zeros(batch_size)))

        d_loss = discriminator.train_on_batch(X_combined, y_combined)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones(batch_size))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

train_gan()

# Generate balanced dataset using GAN
num_samples_to_generate = abs(np.sum(y_train == 0) - np.sum(y_train == 1))
noise = np.random.normal(0, 1, (num_samples_to_generate, latent_dim))
synthetic_data = generator.predict(noise)
synthetic_labels = np.ones(num_samples_to_generate)  # Generate only class 1 samples

X_train_balanced = np.vstack((X_train, synthetic_data))
y_train_balanced = np.hstack((y_train, synthetic_labels))

# Save balanced dataset
balanced_df = pd.DataFrame(X_train_balanced, columns=df.columns[:-1])
balanced_df["F"] = y_train_balanced
balanced_df.to_csv("/content/d7_balancedwithGRU.csv", index=False)

# Train final classifier
classifier = tf.keras.Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(X_train_balanced, y_train_balanced, epochs=100, batch_size=64, verbose=1)

# Evaluate model
y_pred = classifier.predict(X_test).round()
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}, F1-score: {f1}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step 


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0, D Loss: 0.6875370144844055, G Loss: 0.6908113956451416
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


148/148 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5668 - loss: 0.7811
Epoch 2/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6658 - loss: 0.5853
Epoch 3/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6659 - loss: 0.5739
Epoch 4/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6806 - loss: 0.5703
Epoch 5/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6922 - loss: 0.5578
Epoch 6/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6860 - loss: 0.5591
Epoch 7/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6949 - loss: 0.5541
Epoch 8/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6968 - loss: 0.5545
Epoch 9/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6887 - loss: 0.5556
Epoch 10/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6884 - loss: 0.5564
Epoch 11/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6931 - loss: 0.5510
Epoch 12/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step